In [1]:
from keras.models import Sequential
from keras.layers import *
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 160000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '-',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    #print(a, b)
    if int(a) < int(b):
        temp = a
        a = b
        b = temp
    key = tuple((a, b))
    #print(key)
    if key in seen:
        continue
    seen.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 160000


In [8]:
print(questions[:5], expected[:5])

['442-0  ', '601-512', '7-0    ', '87-2   ', '201-5  '] ['442 ', '89  ', '7   ', '85  ', '196 ']


# Processing

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:80000]
train_y = y[:80000]
test_x = x[80000:]
test_y = y[80000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(72000, 7, 12)
(72000, 4, 12)
Validation Data:
(8000, 7, 12)
(8000, 4, 12)
Testing Data:
(80000, 7, 12)
(80000, 4, 12)


In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False  True False False
   False]
  [False False False False False False False False False False False
    True]
  [False False False False False False False  True False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [ True False False False False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False  True False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False False False False False False False False False False  True
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False False False Fal

# Build Model

In [29]:
print('Build model...')

############################################
##### Build your own model here ############
############################################

model = Sequential()
# model.add(Dense(units=256, input_shape=(7, 12), activation='sigmoid'))
# model.add(Flatten())
# model.add(Dense(units=64,activation='relu'))
# model.add(Dense(units=32,activation='relu'))
# model.add(Dense(units=16,activation='relu'))
# model.add(Dense(units=48,activation='softmax'))
# model.add(Reshape((4, 12)))

model.add(LSTM(units=256, input_shape=(7, 12), activation='tanh'))
model.add(RepeatVector(4))
model.add(LSTM(units=128, activation='tanh', return_sequences=True))
model.add(Dense(units=128, activation='sigmoid'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=12, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam')

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 256)               275456    
_________________________________________________________________
repeat_vector_9 (RepeatVecto (None, 4, 256)            0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 4, 128)            197120    
_________________________________________________________________
dense_18 (Dense)             (None, 4, 128)            16512     
_________________________________________________________________
dense_19 (Dense)             (None, 4, 64)             8256      
_________________________________________________________________
dense_20 (Dense)             (None, 4, 12)             780       
Total params: 498,124
Trainable params: 498,124
Non-trainable params: 0
_______________________________________________________

# Training

In [30]:
for iteration in range(50):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 22s 304us/step - loss: 1.7247 - val_loss: 1.6343
Q 810-33  T 777  ☒ 270 
Q 757-523 T 234  ☒ 13  
Q 769-502 T 267  ☒ 13  
Q 761-108 T 653  ☒ 270 
Q 568-56  T 512  ☒ 123 
Q 685-16  T 669  ☒ 220 
Q 755-730 T 25   ☒ 13  
Q 637-59  T 578  ☒ 120 
Q 397-288 T 109  ☒ 13  
Q 962-429 T 533  ☒ 290 

--------------------------------------------------
Iteration 1
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 19s 262us/step - loss: 1.5296 - val_loss: 1.3524
Q 772-210 T 562  ☒ 522 
Q 254-207 T 47   ☒ 1   
Q 916-393 T 523  ☒ 572 
Q 441-77  T 364  ☒ 372 
Q 836-439 T 397  ☒ 472 
Q 947-310 T 637  ☒ 622 
Q 603-61  T 542  ☒ 522 
Q 415-25  T 390  ☒ 322 
Q 429-10  T 419  ☒ 422 
Q 504-474 T 30   ☒ 1   

--------------------------------------------------
Iteration 2
Train on 72000 sam

72000/72000 [==============================] - 19s 259us/step - loss: 0.0324 - val_loss: 0.0269
Q 809-39  T 770  ☑ 770 
Q 469-200 T 269  ☑ 269 
Q 687-209 T 478  ☑ 478 
Q 278-202 T 76   ☑ 76  
Q 895-827 T 68   ☑ 68  
Q 925-20  T 905  ☑ 905 
Q 124-110 T 14   ☒ 24  
Q 346-65  T 281  ☑ 281 
Q 437-28  T 409  ☑ 409 
Q 426-65  T 361  ☑ 361 

--------------------------------------------------
Iteration 16
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 18s 256us/step - loss: 0.0558 - val_loss: 0.0548
Q 955-374 T 581  ☑ 581 
Q 313-222 T 91   ☒ 90  
Q 158-28  T 130  ☑ 130 
Q 818-72  T 746  ☑ 746 
Q 757-554 T 203  ☑ 203 
Q 775-550 T 225  ☑ 225 
Q 757-85  T 672  ☑ 672 
Q 645-245 T 400  ☑ 400 
Q 750-646 T 104  ☑ 104 
Q 588-78  T 510  ☑ 510 

--------------------------------------------------
Iteration 17
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 21s 293us/step - loss: 0.0217 - v

72000/72000 [==============================] - 19s 258us/step - loss: 0.0173 - val_loss: 0.0037
Q 507-84  T 423  ☑ 423 
Q 529-243 T 286  ☑ 286 
Q 647-543 T 104  ☑ 104 
Q 635-115 T 520  ☑ 520 
Q 104-34  T 70   ☑ 70  
Q 829-71  T 758  ☑ 758 
Q 435-325 T 110  ☑ 110 
Q 559-542 T 17   ☑ 17  
Q 815-757 T 58   ☑ 58  
Q 437-55  T 382  ☑ 382 

--------------------------------------------------
Iteration 31
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 19s 261us/step - loss: 0.0024 - val_loss: 0.0040
Q 900-287 T 613  ☑ 613 
Q 471-34  T 437  ☑ 437 
Q 563-225 T 338  ☑ 338 
Q 210-92  T 118  ☑ 118 
Q 504-474 T 30   ☑ 30  
Q 211-70  T 141  ☑ 141 
Q 254-53  T 201  ☑ 201 
Q 377-34  T 343  ☑ 343 
Q 346-312 T 34   ☑ 34  
Q 960-96  T 864  ☑ 864 

--------------------------------------------------
Iteration 32
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 19s 258us/step - loss: 0.0395 - v

72000/72000 [==============================] - 19s 258us/step - loss: 5.1920e-04 - val_loss: 0.0011
Q 902-75  T 827  ☑ 827 
Q 927-32  T 895  ☑ 895 
Q 901-36  T 865  ☑ 865 
Q 322-60  T 262  ☑ 262 
Q 695-84  T 611  ☑ 611 
Q 277-190 T 87   ☑ 87  
Q 137-98  T 39   ☑ 39  
Q 912-178 T 734  ☑ 734 
Q 794-776 T 18   ☑ 18  
Q 552-262 T 290  ☑ 290 

--------------------------------------------------
Iteration 46
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 19s 258us/step - loss: 0.0425 - val_loss: 0.0047
Q 272-49  T 223  ☑ 223 
Q 603-61  T 542  ☑ 542 
Q 823-386 T 437  ☑ 437 
Q 537-42  T 495  ☑ 495 
Q 273-115 T 158  ☑ 158 
Q 666-237 T 429  ☑ 429 
Q 383-88  T 295  ☑ 295 
Q 516-117 T 399  ☑ 399 
Q 362-37  T 325  ☑ 325 
Q 753-107 T 646  ☑ 646 

--------------------------------------------------
Iteration 47
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 19s 258us/step - loss: 0.0016

# Testing

In [31]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
#####################################################
    

MSG : Prediction


In [33]:
test_score = model.evaluate(test_x, test_y)

print("test score :", test_score)

80000/80000 [==============================] - 28s 346us/step
test score : 0.0011383196992537706


In [35]:
output = model.predict_classes(test_x)

In [45]:
count = 0
for i in range(len(test_y[:10000])):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(output[i], calc_argmax=False)
    print(i, 'Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        count += 1
        print(colors.ok + '☑' + colors.close, end=' ')
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)

print("Accuracy : ", count/len(test_y[:10000]))

0 Q 526-505 T 21   ☑ 21  
1 Q 895-72  T 823  ☑ 823 
2 Q 181-37  T 144  ☑ 144 
3 Q 798-142 T 656  ☑ 656 
4 Q 802-684 T 118  ☑ 118 
5 Q 857-16  T 841  ☑ 841 
6 Q 263-76  T 187  ☑ 187 
7 Q 840-737 T 103  ☑ 103 
8 Q 328-54  T 274  ☑ 274 
9 Q 635-73  T 562  ☑ 562 
10 Q 186-8   T 178  ☑ 178 
11 Q 910-675 T 235  ☑ 235 
12 Q 535-129 T 406  ☑ 406 
13 Q 709-341 T 368  ☑ 368 
14 Q 396-57  T 339  ☑ 339 
15 Q 752-40  T 712  ☑ 712 
16 Q 472-160 T 312  ☑ 312 
17 Q 673-18  T 655  ☑ 655 
18 Q 658-95  T 563  ☑ 563 
19 Q 814-557 T 257  ☑ 257 
20 Q 622-425 T 197  ☑ 197 
21 Q 425-247 T 178  ☑ 178 
22 Q 269-63  T 206  ☑ 206 
23 Q 825-408 T 417  ☑ 417 
24 Q 384-132 T 252  ☑ 252 
25 Q 564-216 T 348  ☑ 348 
26 Q 955-200 T 755  ☑ 755 
27 Q 874-31  T 843  ☑ 843 
28 Q 583-94  T 489  ☑ 489 
29 Q 861-83  T 778  ☑ 778 
30 Q 816-786 T 30   ☑ 30  
31 Q 893-397 T 496  ☑ 496 
32 Q 735-554 T 181  ☑ 181 
33 Q 467-132 T 335  ☑ 335 
34 Q 462-5   T 457  ☑ 457 
35 Q 846-220 T 626  ☑ 626 
36 Q 794-54  T 740  ☑ 740 
37 Q 176-78

317 Q 877-22  T 855  ☑ 855 
318 Q 436-186 T 250  ☑ 250 
319 Q 565-43  T 522  ☑ 522 
320 Q 484-172 T 312  ☑ 312 
321 Q 995-27  T 968  ☑ 968 
322 Q 371-97  T 274  ☑ 274 
323 Q 784-46  T 738  ☑ 738 
324 Q 792-394 T 398  ☑ 398 
325 Q 621-369 T 252  ☑ 252 
326 Q 600-55  T 545  ☑ 545 
327 Q 758-463 T 295  ☑ 295 
328 Q 97-53   T 44   ☑ 44  
329 Q 728-423 T 305  ☑ 305 
330 Q 671-91  T 580  ☑ 580 
331 Q 317-288 T 29   ☑ 29  
332 Q 116-25  T 91   ☑ 91  
333 Q 448-47  T 401  ☑ 401 
334 Q 740-59  T 681  ☑ 681 
335 Q 266-15  T 251  ☑ 251 
336 Q 180-90  T 90   ☑ 90  
337 Q 949-626 T 323  ☑ 323 
338 Q 725-58  T 667  ☑ 667 
339 Q 956-211 T 745  ☑ 745 
340 Q 607-16  T 591  ☑ 591 
341 Q 738-37  T 701  ☑ 701 
342 Q 603-26  T 577  ☑ 577 
343 Q 397-203 T 194  ☑ 194 
344 Q 885-694 T 191  ☑ 191 
345 Q 544-105 T 439  ☑ 439 
346 Q 372-189 T 183  ☑ 183 
347 Q 633-437 T 196  ☑ 196 
348 Q 843-351 T 492  ☑ 492 
349 Q 976-33  T 943  ☑ 943 
350 Q 698-34  T 664  ☑ 664 
351 Q 952-8   T 944  ☑ 944 
352 Q 378-19  T 359 

603 Q 590-502 T 88   ☑ 88  
604 Q 663-515 T 148  ☑ 148 
605 Q 666-127 T 539  ☑ 539 
606 Q 771-299 T 472  ☑ 472 
607 Q 841-669 T 172  ☑ 172 
608 Q 518-486 T 32   ☑ 32  
609 Q 466-15  T 451  ☑ 451 
610 Q 440-131 T 309  ☑ 309 
611 Q 675-54  T 621  ☑ 621 
612 Q 725-19  T 706  ☑ 706 
613 Q 717-130 T 587  ☑ 587 
614 Q 738-40  T 698  ☑ 698 
615 Q 806-758 T 48   ☑ 48  
616 Q 90-6    T 84   ☑ 84  
617 Q 945-392 T 553  ☑ 553 
618 Q 636-64  T 572  ☑ 572 
619 Q 334-332 T 2    ☑ 2   
620 Q 801-98  T 703  ☑ 703 
621 Q 839-826 T 13   ☑ 13  
622 Q 495-183 T 312  ☑ 312 
623 Q 965-85  T 880  ☑ 880 
624 Q 530-20  T 510  ☑ 510 
625 Q 228-5   T 223  ☑ 223 
626 Q 693-65  T 628  ☑ 628 
627 Q 693-674 T 19   ☑ 19  
628 Q 806-482 T 324  ☑ 324 
629 Q 821-589 T 232  ☑ 232 
630 Q 599-92  T 507  ☑ 507 
631 Q 955-84  T 871  ☑ 871 
632 Q 850-95  T 755  ☑ 755 
633 Q 623-84  T 539  ☑ 539 
634 Q 109-101 T 8    ☑ 8   
635 Q 865-56  T 809  ☑ 809 
636 Q 857-67  T 790  ☑ 790 
637 Q 804-577 T 227  ☑ 227 
638 Q 386-70  T 316 

960 Q 412-275 T 137  ☑ 137 
961 Q 202-65  T 137  ☑ 137 
962 Q 646-131 T 515  ☑ 515 
963 Q 405-328 T 77   ☑ 77  
964 Q 69-29   T 40   ☑ 40  
965 Q 532-148 T 384  ☑ 384 
966 Q 205-178 T 27   ☑ 27  
967 Q 989-8   T 981  ☑ 981 
968 Q 760-650 T 110  ☑ 110 
969 Q 996-8   T 988  ☑ 988 
970 Q 891-880 T 11   ☑ 11  
971 Q 826-496 T 330  ☑ 330 
972 Q 924-451 T 473  ☑ 473 
973 Q 257-85  T 172  ☑ 172 
974 Q 702-549 T 153  ☑ 153 
975 Q 787-60  T 727  ☑ 727 
976 Q 290-57  T 233  ☑ 233 
977 Q 845-507 T 338  ☑ 338 
978 Q 905-43  T 862  ☑ 862 
979 Q 27-26   T 1    ☑ 1   
980 Q 991-85  T 906  ☑ 906 
981 Q 620-44  T 576  ☑ 576 
982 Q 612-47  T 565  ☑ 565 
983 Q 976-59  T 917  ☑ 917 
984 Q 998-600 T 398  ☑ 398 
985 Q 884-44  T 840  ☑ 840 
986 Q 328-75  T 253  ☑ 253 
987 Q 202-41  T 161  ☑ 161 
988 Q 212-63  T 149  ☑ 149 
989 Q 786-207 T 579  ☑ 579 
990 Q 602-200 T 402  ☑ 402 
991 Q 597-424 T 173  ☑ 173 
992 Q 991-783 T 208  ☑ 208 
993 Q 331-12  T 319  ☑ 319 
994 Q 872-662 T 210  ☑ 210 
995 Q 836-676 T 160 

1317 Q 633-58  T 575  ☑ 575 
1318 Q 742-617 T 125  ☑ 125 
1319 Q 831-151 T 680  ☑ 680 
1320 Q 359-72  T 287  ☑ 287 
1321 Q 609-202 T 407  ☑ 407 
1322 Q 169-97  T 72   ☑ 72  
1323 Q 878-185 T 693  ☑ 693 
1324 Q 786-326 T 460  ☑ 460 
1325 Q 579-4   T 575  ☑ 575 
1326 Q 58-3    T 55   ☑ 55  
1327 Q 853-287 T 566  ☑ 566 
1328 Q 725-215 T 510  ☑ 510 
1329 Q 606-56  T 550  ☑ 550 
1330 Q 425-358 T 67   ☑ 67  
1331 Q 149-2   T 147  ☑ 147 
1332 Q 331-41  T 290  ☑ 290 
1333 Q 713-190 T 523  ☑ 523 
1334 Q 901-311 T 590  ☑ 590 
1335 Q 519-507 T 12   ☑ 12  
1336 Q 724-72  T 652  ☑ 652 
1337 Q 470-256 T 214  ☑ 214 
1338 Q 78-4    T 74   ☑ 74  
1339 Q 875-66  T 809  ☑ 809 
1340 Q 755-77  T 678  ☑ 678 
1341 Q 578-518 T 60   ☑ 60  
1342 Q 78-64   T 14   ☑ 14  
1343 Q 665-31  T 634  ☑ 634 
1344 Q 633-598 T 35   ☑ 35  
1345 Q 180-83  T 97   ☑ 97  
1346 Q 605-82  T 523  ☑ 523 
1347 Q 951-5   T 946  ☑ 946 
1348 Q 778-246 T 532  ☑ 532 
1349 Q 817-553 T 264  ☑ 264 
1350 Q 726-75  T 651  ☑ 651 
1351 Q 690-614

...

8316 Q 508-474 T 34   ☑ 34  
8317 Q 892-400 T 492  ☑ 492 
8318 Q 916-853 T 63   ☑ 63  
8319 Q 660-465 T 195  ☑ 195 
8320 Q 988-96  T 892  ☑ 892 
8321 Q 731-420 T 311  ☑ 311 
8322 Q 551-43  T 508  ☑ 508 
8323 Q 764-632 T 132  ☑ 132 
8324 Q 126-56  T 70   ☑ 70  
8325 Q 935-585 T 350  ☑ 350 
8326 Q 795-396 T 399  ☑ 399 
8327 Q 899-266 T 633  ☑ 633 
8328 Q 563-20  T 543  ☑ 543 
8329 Q 508-23  T 485  ☑ 485 
8330 Q 870-38  T 832  ☑ 832 
8331 Q 462-1   T 461  ☑ 461 
8332 Q 956-410 T 546  ☑ 546 
8333 Q 633-26  T 607  ☑ 607 
8334 Q 827-4   T 823  ☑ 823 
8335 Q 343-298 T 45   ☑ 45  
8336 Q 192-5   T 187  ☑ 187 
8337 Q 662-96  T 566  ☑ 566 
8338 Q 606-4   T 602  ☑ 602 
8339 Q 430-113 T 317  ☑ 317 
8340 Q 245-70  T 175  ☑ 175 
8341 Q 602-28  T 574  ☑ 574 
8342 Q 549-265 T 284  ☑ 284 
8343 Q 767-665 T 102  ☑ 102 
8344 Q 686-194 T 492  ☑ 492 
8345 Q 627-159 T 468  ☑ 468 
8346 Q 510-86  T 424  ☑ 424 
8347 Q 970-24  T 946  ☑ 946 
8348 Q 776-147 T 629  ☑ 629 
8349 Q 517-374 T 143  ☑ 143 
8350 Q 389-215

8673 Q 784-605 T 179  ☑ 179 
8674 Q 822-256 T 566  ☑ 566 
8675 Q 238-60  T 178  ☑ 178 
8676 Q 806-66  T 740  ☑ 740 
8677 Q 943-260 T 683  ☑ 683 
8678 Q 311-73  T 238  ☑ 238 
8679 Q 795-17  T 778  ☑ 778 
8680 Q 317-86  T 231  ☑ 231 
8681 Q 637-6   T 631  ☑ 631 
8682 Q 907-43  T 864  ☑ 864 
8683 Q 561-414 T 147  ☑ 147 
8684 Q 849-14  T 835  ☑ 835 
8685 Q 474-73  T 401  ☑ 401 
8686 Q 323-29  T 294  ☑ 294 
8687 Q 757-28  T 729  ☑ 729 
8688 Q 633-129 T 504  ☑ 504 
8689 Q 270-127 T 143  ☑ 143 
8690 Q 593-64  T 529  ☑ 529 
8691 Q 865-144 T 721  ☑ 721 
8692 Q 762-394 T 368  ☑ 368 
8693 Q 837-95  T 742  ☑ 742 
8694 Q 475-269 T 206  ☑ 206 
8695 Q 784-40  T 744  ☑ 744 
8696 Q 834-668 T 166  ☑ 166 
8697 Q 468-80  T 388  ☑ 388 
8698 Q 801-580 T 221  ☑ 221 
8699 Q 848-84  T 764  ☑ 764 
8700 Q 740-149 T 591  ☑ 591 
8701 Q 166-34  T 132  ☑ 132 
8702 Q 660-600 T 60   ☑ 60  
8703 Q 962-849 T 113  ☑ 113 
8704 Q 851-713 T 138  ☑ 138 
8705 Q 515-66  T 449  ☑ 449 
8706 Q 314-72  T 242  ☑ 242 
8707 Q 149-13 

9017 Q 700-314 T 386  ☑ 386 
9018 Q 606-396 T 210  ☑ 210 
9019 Q 66-65   T 1    ☑ 1   
9020 Q 90-64   T 26   ☑ 26  
9021 Q 674-68  T 606  ☑ 606 
9022 Q 703-209 T 494  ☑ 494 
9023 Q 58-49   T 9    ☑ 9   
9024 Q 198-0   T 198  ☑ 198 
9025 Q 623-19  T 604  ☑ 604 
9026 Q 879-261 T 618  ☑ 618 
9027 Q 953-21  T 932  ☑ 932 
9028 Q 197-78  T 119  ☑ 119 
9029 Q 408-64  T 344  ☑ 344 
9030 Q 99-38   T 61   ☑ 61  
9031 Q 391-39  T 352  ☑ 352 
9032 Q 561-196 T 365  ☑ 365 
9033 Q 929-220 T 709  ☑ 709 
9034 Q 689-532 T 157  ☑ 157 
9035 Q 388-150 T 238  ☑ 238 
9036 Q 993-240 T 753  ☑ 753 
9037 Q 962-508 T 454  ☑ 454 
9038 Q 974-8   T 966  ☑ 966 
9039 Q 650-472 T 178  ☑ 178 
9040 Q 799-761 T 38   ☑ 38  
9041 Q 330-160 T 170  ☑ 170 
9042 Q 362-141 T 221  ☑ 221 
9043 Q 576-85  T 491  ☑ 491 
9044 Q 746-526 T 220  ☑ 220 
9045 Q 301-292 T 9    ☒ 1   
9046 Q 429-145 T 284  ☑ 284 
9047 Q 103-76  T 27   ☑ 27  
9048 Q 220-13  T 207  ☑ 207 
9049 Q 936-12  T 924  ☑ 924 
9050 Q 609-449 T 160  ☑ 160 
9051 Q 957-266

9379 Q 351-314 T 37   ☑ 37  
9380 Q 759-24  T 735  ☑ 735 
9381 Q 883-67  T 816  ☑ 816 
9382 Q 118-84  T 34   ☑ 34  
9383 Q 745-20  T 725  ☑ 725 
9384 Q 951-87  T 864  ☑ 864 
9385 Q 907-66  T 841  ☑ 841 
9386 Q 406-91  T 315  ☑ 315 
9387 Q 529-37  T 492  ☑ 492 
9388 Q 483-56  T 427  ☑ 427 
9389 Q 473-220 T 253  ☑ 253 
9390 Q 583-36  T 547  ☑ 547 
9391 Q 186-31  T 155  ☑ 155 
9392 Q 573-40  T 533  ☑ 533 
9393 Q 625-415 T 210  ☑ 210 
9394 Q 263-88  T 175  ☑ 175 
9395 Q 75-68   T 7    ☑ 7   
9396 Q 695-573 T 122  ☑ 122 
9397 Q 447-51  T 396  ☑ 396 
9398 Q 944-668 T 276  ☑ 276 
9399 Q 618-370 T 248  ☑ 248 
9400 Q 887-85  T 802  ☑ 802 
9401 Q 722-121 T 601  ☑ 601 
9402 Q 892-768 T 124  ☑ 124 
9403 Q 703-79  T 624  ☑ 624 
9404 Q 893-670 T 223  ☑ 223 
9405 Q 920-571 T 349  ☑ 349 
9406 Q 671-52  T 619  ☑ 619 
9407 Q 664-79  T 585  ☑ 585 
9408 Q 519-325 T 194  ☑ 194 
9409 Q 628-94  T 534  ☑ 534 
9410 Q 184-90  T 94   ☑ 94  
9411 Q 260-84  T 176  ☑ 176 
9412 Q 981-895 T 86   ☑ 86  
9413 Q 269-5  

9712 Q 248-88  T 160  ☑ 160 
9713 Q 425-41  T 384  ☑ 384 
9714 Q 799-1   T 798  ☑ 798 
9715 Q 927-13  T 914  ☑ 914 
9716 Q 906-838 T 68   ☑ 68  
9717 Q 91-79   T 12   ☑ 12  
9718 Q 304-16  T 288  ☑ 288 
9719 Q 961-40  T 921  ☑ 921 
9720 Q 650-374 T 276  ☑ 276 
9721 Q 137-94  T 43   ☑ 43  
9722 Q 940-503 T 437  ☑ 437 
9723 Q 467-111 T 356  ☑ 356 
9724 Q 81-66   T 15   ☑ 15  
9725 Q 166-48  T 118  ☑ 118 
9726 Q 729-67  T 662  ☑ 662 
9727 Q 514-96  T 418  ☑ 418 
9728 Q 916-18  T 898  ☑ 898 
9729 Q 601-4   T 597  ☑ 597 
9730 Q 314-47  T 267  ☑ 267 
9731 Q 63-13   T 50   ☑ 50  
9732 Q 496-204 T 292  ☑ 292 
9733 Q 653-182 T 471  ☑ 471 
9734 Q 653-453 T 200  ☑ 200 
9735 Q 580-284 T 296  ☑ 296 
9736 Q 358-87  T 271  ☑ 271 
9737 Q 947-12  T 935  ☑ 935 
9738 Q 976-406 T 570  ☑ 570 
9739 Q 876-93  T 783  ☑ 783 
9740 Q 267-98  T 169  ☑ 169 
9741 Q 765-173 T 592  ☑ 592 
9742 Q 866-325 T 541  ☑ 541 
9743 Q 703-92  T 611  ☑ 611 
9744 Q 646-88  T 558  ☑ 558 
9745 Q 659-31  T 628  ☑ 628 
9746 Q 448-82 